In [1]:
import numpy as np

def tsne_visualize1(alpha,rho,beta,topwords=10):
    alpha_rho=np.vstack((alpha, rho))
    tsne_model = TSNE(n_components=2, verbose=0, random_state=0, angle=.99, init='pca')
    tsne_lda = tsne_model.fit_transform(alpha_rho)
    topK=np.argsort(-1 * beta, axis=1)[:,0:topwords]
    topKvect=np.concatenate(topK)
    indexes = list(range(50))
    for i in list(topKvect):
        indexes.append(i)
    tsne_lda_topK=tsne_lda[indexes,]
    
    topics=alpha.shape[0]
    colours=list(range(topics))
    markers=list(np.repeat('D',topics))
    for i in range(topics):
        for j in range(K):
            colours.append(i)
            markers.append('.')
    
    colormap = []
    for name, hex in matplotlib.colors.cnames.items():
        colormap.append(hex)
    colormap = colormap[:tsne_lda_topK.shape[0]]
    colormap = list(np.array(colormap))
    colours2 = [colormap[c] for c in colours]
    
    fig = plt.figure()
    ax1 = fig.add_subplot(111)
    ax1.scatter(x=tsne_lda_topK[topics:, 0], y=tsne_lda_topK[topics:, 1], c=colours[topics:], marker='.', label='first')
    ax1.scatter(x=tsne_lda_topK[:topics, 0], y=tsne_lda_topK[:topics, 1], c=colours[:topics], marker='D', label='second')
    #plt.legend(loc='upper left')
    #plt.xlim(-2, 2.5)
    #plt.ylim(-3, 3)
    plt.show()

In [2]:
import seaborn as sns
import pandas as pd

def barplot_topwords(beta, vocab, topic, topwords=10):
    topK=np.argsort(-1 * beta[topic,])[0:topwords]
    data = {'word':[str(vocab[i]) for i in list(topK)],'beta':list(beta[topic,topK])}
    df = pd.DataFrame(data)
    sns.barplot(x='beta', y='word', data = df, color='black', orient='h')


In [3]:
top_topics=list(np.argsort(-1 * theta, axis=1)[:,1])
theta_hist=sns.histplot(top_topics).set(xlabel='θ', ylabel='counts')
plt.show()


NameError: name 'theta' is not defined